In [11]:
import os
import json
import pandas as pd
import traceback

In [12]:
!pip install langchain-community
!pip install --upgrade langchain
!pip show langchain-community

Name: langchain-community
Version: 0.3.0
Summary: Community contributed LangChain integrations.
Home-page: https://github.com/langchain-ai/langchain
Author: 
Author-email: 
License: MIT
Location: c:\users\armst\mcqgen\env\lib\site-packages
Requires: aiohttp, dataclasses-json, langchain, langchain-core, langsmith, numpy, pydantic-settings, PyYAML, requests, SQLAlchemy, tenacity
Required-by: 


In [13]:
from langchain.chat_models import ChatOpenAI

In [14]:
from dotenv import load_dotenv
load_dotenv()

True

In [15]:
KEY=os.getenv("OPENAI_API_KEY")

In [16]:
print(KEY)

sk-30gfZtow6BxC9sbSSrMAFgS0-afdbHxYsUHhVHuryIT3BlbkFJzeWL8U7weTvfaKe_jK1wKyQ1IafMgmSmKqkTYzcn8A


In [17]:
llm=ChatOpenAI(openai_api_key=KEY,model_name="gpt-3.5-turbo",temperature=0.5)

C:\Users\armst\AppData\Local\Temp\ipykernel_27796\3870565440.py:1: LangChainDeprecationWarning: The class `ChatOpenAI` was deprecated in LangChain 0.0.10 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import ChatOpenAI``.
  llm=ChatOpenAI(openai_api_key=KEY,model_name="gpt-3.5-turbo",temperature=0.5)


In [18]:
llm

ChatOpenAI(client=<openai.resources.chat.completions.Completions object at 0x000002C46B5EDF30>, async_client=<openai.resources.chat.completions.AsyncCompletions object at 0x000002C46B624070>, temperature=0.5, model_kwargs={}, openai_api_key='sk-30gfZtow6BxC9sbSSrMAFgS0-afdbHxYsUHhVHuryIT3BlbkFJzeWL8U7weTvfaKe_jK1wKyQ1IafMgmSmKqkTYzcn8A', openai_proxy='')

In [20]:
from langchain.llms import OpenAI
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain
from langchain.callbacks import get_openai_callback
import PyPDF2

In [21]:
RESPONSE_JSON = {
    "1":{
        "mcq": "multiple choice question",
        "options": {
            "a": "choice here",
            "b": "choice here",
            "c": "choice here",
            "d": "choice here",
        },
    "correct": "correct answer",
    },
    "2":{
        "mcq": "multiple choice question",
        "options": {
            "a": "choice here",
            "b": "choice here",
            "c": "choice here",
            "d": "choice here",
        },
    "correct": "correct answer",
    },
    "3":{
        "mcq": "multiple choice question",
        "options": {
            "a": "choice here",
            "b": "choice here",
            "c": "choice here",
            "d": "choice here",
        },
    "correct": "correct answer",
    },
}

In [22]:
TEMPLATE="""
Text:{text}
You are an expert MCQ maker. Given the above text, it is your job to \
create a quiz of {number} multiple choice question for {subject} student in {tone} tone.
Make sure the question are not repeated and check all the questions to be conforming the text as well.
Make sure to format your response like RESPONSE_JSON below and use it as a guide. \
Ensure to make {number} MCQs
### RESPONSE_JSON
{response_json}

"""

In [24]:
quiz_generation_prompt = PromptTemplate(
    import_variables=["text", "number", "subject", "tone", "response_json"],
    template=TEMPLATE
)

In [25]:
quiz_chain=LLMChain(llm=llm,prompt=quiz_generation_prompt, output_key="quiz",verbose=True)

C:\Users\armst\AppData\Local\Temp\ipykernel_27796\1166302129.py:1: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 1.0. Use :meth:`~RunnableSequence, e.g., `prompt | llm`` instead.
  quiz_chain=LLMChain(llm=llm,prompt=quiz_generation_prompt, output_key="quiz",verbose=True)


In [26]:
TEMPLATE2="""
You are an expert english grammarian and writer. Given a Multiple Choice Quiz for {subject} students.\
You need to evaluate the complexity of the question and give a complex analysis of the quiz. Only use at max 50 words for complexity
if the quiz is not at per with the cognitive and analytical abilities of the students, \
update the quiz question which needs to be changes and change the tone such that it perfectly fits the student abilities
Quiz_MCQs:
{quiz}

Check from an expert English Writer of the above quiz:
"""

In [34]:
quiz_evaluation_prompt=PromptTemplate(input_variables=["subject", "quiz"], template=TEMPLATE)

In [35]:
review_chain=LLMChain(llm=llm, prompt=quiz_evaluation_prompt, output_key="review", verbose=True)

In [46]:
generate_evaluate_chain=SequentialChain(chains=[quiz_chain, review_chain], input_variables=["text", "number", "subject", "tone", "response_json"],
                                        output_variables=["quiz", "review"], verbose=True,)


ValidationError: 1 validation error for SequentialChain
  Value error, Missing required input keys: {'numberrr'}, only had {'response_json', 'subject', 'tone', 'number', 'text'} [type=value_error, input_value={'chains': [LLMChain(verb...view'], 'verbose': True}, input_type=dict]
    For further information visit https://errors.pydantic.dev/2.9/v/value_error

In [48]:
file_path=r"C:\Users\armst\mcqgen\experiment\data.txt"

In [49]:
with open(file_path, 'r') as file:
    TEXT = file.read()

In [52]:
print(TEXT)

In [53]:
#serialize the python dictionary into a JSON-formatted string
json.dumps(RESPONSE_JSON)

'{"1": {"mcq": "multiple choice question", "options": {"a": "choice here", "b": "choice here", "c": "choice here", "d": "choice here"}, "correct": "correct answer"}, "2": {"mcq": "multiple choice question", "options": {"a": "choice here", "b": "choice here", "c": "choice here", "d": "choice here"}, "correct": "correct answer"}, "3": {"mcq": "multiple choice question", "options": {"a": "choice here", "b": "choice here", "c": "choice here", "d": "choice here"}, "correct": "correct answer"}}'

In [54]:
NUMBER = 5
SUBJECT = "machine learning"
TONE = "simple"

In [55]:
#how to setup token usage tracking in Langchain
with get_openai_callback() as cb:
    response=generate_evaluate_chain(
    {
        "text": TEXT,
        "number": NUMBER,
        "subject": SUBJECT,
        "tone": TONE,
        "response_json": json.dumps(RESPONSE_JSON)
    }
    )

NameError: name 'generate_evaluate_chain' is not defined

In [ ]:
print(f"Total Tokens:{cb.total_tokens}")
print(f"Prompt Tokens:{cb.prompt_tokens}")
print(f"Completion Tokens:{cb.completion_tokens}")
print(f"Total cost:{cb.total_cost}")

In [ ]:
response

In [ ]:
quiz = response.get("quiz")

In [ ]:
quiz = json.loads(quiz)

In [ ]:
quiz_table_data = []
for key, value in quiz.items():
    mcq = value["mcq"]
    options = " | ".json(
        [
            f"{option}: {option_value}"
            for option, option_value in value["options"].items() 
            ]
        )
    correct = value["correct"]
    quiz_table_data.append({"MCQ": mcq, "Choices: options, "Correct": correct})

In [ ]:
quiz_table_data

In [ ]:
quiz = pd.DataFrame(quiz_table_data)

In [ ]:
quiz.to_csv("machine_learning.csv",index=False)